In [1]:
pip install 'portalocker>=2.0.0'

In [2]:
pip install torchinfo

In [3]:
import torch
import torchtext
import pandas as pd
from torchinfo import summary
#from torchnlp import *
#train_dataset, test_dataset, classes, vocab = load_dataset()
from pdb import set_trace as bp  # added break point accessor####################
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer("basic_english")

def listToString(s):

    # initialize an empty string
    str1 = " "
    #bp()

    # return string
    return (str1.join(s))


#vocab_size = len(vocab)

class NLP_dataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, input_text_file, transform=None):
        """
        Arguments:
            input_text_file (string): Path to the input file with annotations.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        #self.data_frame = pd.read_csv(input_text_file, header=None, quotechar='"')
        self.data_frame = pd.read_csv(input_text_file, header=None, quotechar=None,quoting=3, on_bad_lines='skip')

        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        #bp()

        label = torch.tensor(int(self.data_frame.iloc[idx, 0].replace('"', '')), dtype=torch.long)
        text = self.data_frame.iloc[idx, 1:].values
        #bp()
        text = text.tolist()
        text = map(str, text)
        text = listToString(text)
        text = text.replace('"', '')


        #input_name = '/home/anurenjanr/BRATS_2019/Data/NPY/input_data_3channels/images/' + input_name
        #target_name = input_name.replace("/images/", "/masks/")
        #target_name = target_name.replace("image_", "mask_")

        #images = np.load(input_name)
        #images_seg = np.load(target_name)

        #bp()

        #if self.transform is not None:
            #image, mask = self.transform((images, images_seg))

        # fix dimensions (C, H, W)
        #image = image.transpose(2, 0, 1)
        #mask = mask.transpose(2, 0, 1)

        #image_tensor = torch.from_numpy(image.astype(np.float32))
        #mask_tensor = torch.from_numpy(mask.astype(np.float32))

        sample = {'label': label, 'text': text}

        return sample

NLP_dataset_train = NLP_dataset(input_text_file='/content/drive/MyDrive/NLP/data/train.csv')
NLP_dataset_test = NLP_dataset(input_text_file='/content/drive/MyDrive/NLP/data/test.csv')

#brats_dataset = brats_2019_train(input_text_file='/home/anurenjanr/BRATS_2019/train_npy_list_random_200.txt', transform=transforms(scale=0.05, angle=None, flip_prob=None))

#bp()

train_dataloader = DataLoader(NLP_dataset_train, batch_size=1, shuffle=False, num_workers=0)
test_dataloader = DataLoader(NLP_dataset_test, batch_size=1, shuffle=False, num_workers=0)

#for i, sample_batched in enumerate(train_dataloader):
    #bp()
   # label, text = sample_batched['label'], sample_batched['text']
    #bp()
def yield_tokens(data_iter):
    for i, sample_batched in enumerate(data_iter):
        label, text = sample_batched['label'], sample_batched['text']
        #bp()
        yield tokenizer(text[0])

vocab = build_vocab_from_iterator(yield_tokens(train_dataloader), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])
vocab(['here', 'is', 'an', 'example'])

#bp()

[437, 26, 31, 5335]

In [9]:
#vocab(['here', 'is', 'an', 'example'])

text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


from torch import nn


class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        #bp()
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

vocab_size = len(vocab)
emsize = 64
num_class = 4

model = TextClassificationModel(vocab_size, emsize, num_class).to(device)


In [6]:

import time


def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, sample_batched in enumerate(dataloader):
        label, text = sample_batched['label'], sample_batched['text']
        #bp()
        processed_text = torch.tensor(text_pipeline(text[0]), dtype=torch.int64)
        offsets = torch.tensor([processed_text.size(0)])
        offsets = torch.tensor([0])

        label = torch.tensor(label_pipeline(label[0]), dtype=torch.int64)
        label = torch.unsqueeze(label, 0)
        optimizer.zero_grad()
        predicted_label = model(processed_text, offsets)
        #bp()
        loss = criterion(predicted_label, label)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print(
                "| epoch {:3d} | {:5d}/{:5d} batches "
                "| accuracy {:8.3f}".format(
                    epoch, idx, len(dataloader), total_acc / total_count
                )
            )
            total_acc, total_count = 0, 0
            start_time = time.time()


In [7]:
def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, sample_batched in enumerate(dataloader):
            #bp()
            label, text = sample_batched['label'], sample_batched['text']
            #processed_text = torch.tensor(text_pipeline(text[0]), dtype=torch.int64)
            processed_text = torch.tensor(text_pipeline(text), dtype=torch.int64)

            offsets = torch.tensor([0])
            label = torch.tensor(label_pipeline(label), dtype=torch.int64)
            label = torch.unsqueeze(label, 0)
            predicted_label = model(processed_text, offsets)
            #bp()
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc / total_count

In [10]:
EPOCHS = 3
LR = 5  # learning rate
total_accu = None
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    #bp()
    accu_val = evaluate(NLP_dataset_test)
    #bp()
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print("-" * 59)
    print(
        "| end of epoch {:3d} | time: {:5.2f}s | "
        "valid accuracy {:8.3f} ".format(
            epoch, time.time() - epoch_start_time, accu_val
        )
    )
    print("-" * 59)

| epoch   1 |   500/12107 batches | accuracy    0.838
| epoch   1 |  1000/12107 batches | accuracy    0.412
| epoch   1 |  1500/12107 batches | accuracy    0.570
| epoch   1 |  2000/12107 batches | accuracy    0.714
| epoch   1 |  2500/12107 batches | accuracy    0.752
| epoch   1 |  3000/12107 batches | accuracy    0.760
| epoch   1 |  3500/12107 batches | accuracy    0.784
| epoch   1 |  4000/12107 batches | accuracy    0.812
| epoch   1 |  4500/12107 batches | accuracy    0.770
| epoch   1 |  5000/12107 batches | accuracy    0.778
| epoch   1 |  5500/12107 batches | accuracy    0.824
| epoch   1 |  6000/12107 batches | accuracy    0.830
| epoch   1 |  6500/12107 batches | accuracy    0.814
| epoch   1 |  7000/12107 batches | accuracy    0.864
| epoch   1 |  7500/12107 batches | accuracy    0.838
| epoch   1 |  8000/12107 batches | accuracy    0.824
| epoch   1 |  8500/12107 batches | accuracy    0.854
| epoch   1 |  9000/12107 batches | accuracy    0.856
| epoch   1 |  9500/12107 ba

In [ ]:
class Test(object):

    # This function prints the type
    # of the object passed as well
    # as the object item
    def __init__(self, items):
        print ("type(items), items")

    def __getitem__(self, items):
        print (type(items), items)

# Driver code
test = Test(5)

type(items), items
